In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
root = os.path.abspath("./drive/MyDrive/data-study/")

In [3]:
csv = os.path.join(root, "state.csv")
data = pd.read_csv(csv)
data[:10]

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ
3,Arkansas,2915918,5.6,AR
4,California,37253956,4.4,CA
5,Colorado,5029196,2.8,CO
6,Connecticut,3574097,2.4,CT
7,Delaware,897934,5.8,DE
8,Florida,18801310,5.8,FL
9,Georgia,9687653,5.7,GA




* **평균(mean)**

  : 모든 관측값을 더해서 관측값 개수로 나눈 값

>
$$
\begin{align}
\overline{x}=\frac{\sum_i^n x_i}{n}
\end{align}
$$

* **절사평균(trimmed mean)**

  : 상위·하위 일정 비율을 제외한 평균  
  $\rightarrow$ 자료에 극단치가 있을 경우 이를 제외할 수 있어 유용함

> 
$$
\begin{align}
\overline{x}=\frac{\sum_{i=p+1}^{n-p} x_{(i)}}{n-2p}
\end{align}
$$

* **중간값(median)**

  : 데이터를 일렬로 정렬했을 때, 한가운데에 위치하는 값

In [4]:
def mean(datas):
  return sum(datas) / len(datas)

def cut_mean(datas, trim:int=5):
  datas = list(datas)
  datas.sort()
  datas = datas[trim: -1*trim]
  return sum(datas) / len(datas)

def median(datas):
  datas = list(datas)
  datas.sort()
  center = len(datas) // 2
  if len(datas) % 2 == 1:
    return datas[center]
  return (datas[center - 1] + datas[center]) / 2

In [5]:
population = data["Population"]
mean(population)

6162876.3

In [6]:
cut_mean(population)

4783697.125

In [7]:
median(population)

4436369.5

In [8]:
def weighted_mean(datas, w):
  return sum(datas * w) / sum(w)

def cut_weighted_mean(datas, trim:int=1):
  datas.sort(key=lambda x: x[0] * x[1])
  return sum(list(map(lambda x: x[0] * x[1],datas[trim: -1*trim]))) / sum(list(map(lambda x: x[1],datas[trim: -1*trim])))

murder_rate = data["Murder.Rate"]
population = data["Population"]

In [9]:
weighted_mean(murder_rate, population)

4.445833981123394

In [10]:
cut_weighted_mean(data[["Murder.Rate", "Population"]].values.tolist(), 1)

4.458740807760504

* **편차(deviation)** 

  : 관측값과 위치 추정값(평균) 사이의 차이 (유의어: 오차, 잔차) 

* **분산(variance)**

  : 편차 제곱의 합을 n-1 로 나눈 값, n은 데이터 개수 (유의어: 평균제곱 오차)

>
$$
\begin{align}
Var=\frac{\sum(x-\overline{x})^2}{n-1}
\end{align}
$$

* **표준편차(standard deviation)**

  : 분산의 제곱근 (유의어: 노름, 유클리드 노름)

> 
$$
\begin{align}
σ=\sqrt{Var}
\end{align}
$$

In [11]:
from math import sqrt

def dev(data):
  return data - mean(data)

def var(data):
  return sum(dev(data) ** 2) / (len(data) - 1)

def sd(data):
  return sqrt(var(data))

In [12]:
dev(data["Population"])

0     -1383140.3
1     -5452645.3
2       229140.7
3     -3246958.3
4     31091079.7
5     -1133680.3
6     -2588779.3
7     -5264942.3
8     12638433.7
9      3524776.7
10    -4802575.3
11    -4595294.3
12     6667755.7
13      320925.7
14    -3116521.3
15    -3309758.3
16    -1823509.3
17    -1629504.3
18    -4834515.3
19     -389324.3
20      384752.7
21     3720763.7
22     -858951.3
23    -3195579.3
24     -173949.3
25    -5173461.3
26    -4336535.3
27    -3462325.3
28    -4846406.3
29     2629017.7
30    -4103697.3
31    13215225.7
32     3372606.7
33    -5490285.3
34     5373627.7
35    -2411525.3
36    -2331802.3
37     6539502.7
38    -5110309.3
39    -1537512.3
40    -5348696.3
41      183228.7
42    18982684.7
43    -3398991.3
44    -5537135.3
45     1838147.7
46      561663.7
47    -4309882.3
48     -475890.3
49    -5599250.3
Name: Population, dtype: float64

In [13]:
var(data["Population"])

46898327373394.445

In [14]:
sd(data["Population"])

6848235.347401142



*   범위
*   백분위수
*   사분위범위(IQR)



In [41]:
# 방법1
def IQR(datas):
  datas = list(datas)
  datas.sort()
  center = len(datas) // 2 
  cc = center // 2  
  if len(datas) % 2 == 1:
    a = datas[:center]
    b = datas[center+1:]
    q1 = a[cc]
    q3 = b[cc]
  else:
    a = datas[:center]
    b = datas[center:]
    q1 = (a[cc-1] + a[cc]) / 2
    q3 = (b[cc-1] + b[cc]) / 2
  return q3 - q1

In [42]:
IQR(population)

4939123.0

In [67]:
# 방법2
def IQR(datas):
  datas = list(datas)
  datas.sort()
  if len(datas) % 2 == 1:
    q1 = datas[int(len(datas) * 1/4)]
    q3 = datas[int(len(datas) * 3/4)]
  else:
    q1 = (datas[int(len(datas) * 1/4)] + datas[int(len(datas) * 1/4) - 1]) / 2
    q3 = (datas[int(len(datas) * 3/4)] + datas[int(len(datas) * 3/4) - 1]) / 2
  return q3 - q1

*   **평균절대편차(AAD/MAD)**

    : 편차의 절댓값의 평균

>
$$
\begin{align}
\frac{\sum_{i=1}^n|x_i-\overline{x}|}{n}
\end{align}
$$


*   **중위절대편차(MAD)**
    
    : 평균절대편차와 유사하지만 평균 대신 중앙값을 쓴다는 차이가 있다.

>
$$
\begin{align}
\frac{\sum_{i=1}^n|x_i-m|}{n}
\end{align}
$$


In [29]:
def mad(data):
  return sum(abs(data - median(data))) / len(data)

In [30]:
mad(data["Population"])

4147655.26